In [1]:
import numpy as np
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point

ROOT_FILES = 'C:/Users/Linus/PycharmProjects/BA/'
ROOT_RESOURCE = 'src/main/resources/QGIS/Nutzungsplan/'

def create_geo_dataframe(geo_files):
    result = []
    for geo_file in geo_files:
        shapefile_path = ROOT_FILES + ROOT_RESOURCE + geo_file
        frame = gpd.read_file(shapefile_path)
        if not frame.empty:  # Prüft, ob das GeoDataFrame nicht leer ist
            result.append(frame)

    if result:
        gdf = gpd.GeoDataFrame(pd.concat(result, ignore_index=True))
    else:
        gdf = gpd.GeoDataFrame()  # Wenn keine Daten vorhanden sind, wird ein leeres GeoDataFrame erstellt

    return gdf


def filter_data(daten, gewuenschtes_jahr, gewuenschte_gemeinden,
                gewuenschte_zielnamen, gewuenschte_kategorien,
                gewuenschte_verkehrsmittel):
    return daten[(daten['jahr'] == gewuenschtes_jahr) &
                 (daten['quelle_name'].isin(gewuenschte_gemeinden)) &
                 (daten['ziel_name'].isin(gewuenschte_zielnamen)) &
                 (daten['kategorie'].isin(gewuenschte_kategorien)) &
                 (daten['verkehrsmittel'].isin(gewuenschte_verkehrsmittel))]

def filter_target_data(daten_filtered, target):
    return daten_filtered[(daten_filtered['quelle_name'] == target)
                          & (daten_filtered['quelle_name']
                             != daten_filtered['ziel_name'])]

def filter_and_calculate_traffic_data(data, gemeinde_quelle, gemeinde_ziel, verkehrsmittel, jahr, kategorie):
    filtered_data = data[
        (data['quelle_name'] == gemeinde_quelle) &
        (data['ziel_name'] == gemeinde_ziel) &
        (data['verkehrsmittel'] == verkehrsmittel) &
        (data['jahr'] == jahr) &
        (data['kategorie'] == kategorie)
    ]
    return filtered_data['wert'].values[0]

def berechne_verkehrsaufkommen(gemeinde_oev, gemeinde_miv, modal_split_miv):
    gesamt_verkehrsaufkommen = gemeinde_oev + gemeinde_miv
    neues_verkehrsaufkommen_oev = gesamt_verkehrsaufkommen * (modal_split_miv / 100)
    return neues_verkehrsaufkommen_oev

def calculate_demand(daten, stunden_verkehrstag, prozent_verteilung_hoch,
                     prozent_verteilung_mittel, prozent_verteilung_niedrig):
    nachfrage = int(daten['wert'].sum() / stunden_verkehrstag)
    #nachfrage_zentral = int(prozent_verteilung_hoch * nachfrage)
    nachfrage_höhere_Dichte = int(prozent_verteilung_mittel * nachfrage)
    nachfrage_niedrige_Dichte = int(prozent_verteilung_niedrig * nachfrage)
    nachfrage_zentral = nachfrage - nachfrage_höhere_Dichte - nachfrage_niedrige_Dichte
    return nachfrage, nachfrage_zentral,\
        nachfrage_höhere_Dichte, nachfrage_niedrige_Dichte

def calculate_new_demand(daten, additional_data, stunden_verkehrstag, prozent_verteilung_hoch,
                     prozent_verteilung_mittel, prozent_verteilung_niedrig):
    nachfrage = int((daten['wert'].sum() + additional_data) / stunden_verkehrstag)
    #nachfrage_zentral = int(prozent_verteilung_hoch * nachfrage)
    nachfrage_höhere_Dichte = int(prozent_verteilung_mittel * nachfrage)
    nachfrage_niedrige_Dichte = int(prozent_verteilung_niedrig * nachfrage)
    nachfrage_zentral = nachfrage - nachfrage_höhere_Dichte - nachfrage_niedrige_Dichte
    return nachfrage, nachfrage_zentral,\
        nachfrage_höhere_Dichte, nachfrage_niedrige_Dichte

def generate_random_timestamp(start, end, n):
    time_diff = (end - start).total_seconds()
    random_seconds = np.random.randint(0, int(time_diff), n)
    return [start + pd.Timedelta(seconds=sec) for sec in random_seconds]
def verteile_nachfragepunkte(gemeinde, nachfrage_zentral, nachfrage_höhere_Dichte,
                             nachfrage_niedrige_Dichte, gemeinden_zentral_gdfs,
                             gemeinden_höhere_Dichte_gdfs, gemeinden_niedrige_Dichte_gdfs,
                             start_timestamp, end_timestamp):
    nachfragepunkte_zentral = np.random.choice(
        gemeinden_zentral_gdfs[gemeinden_zentral_gdfs['GEMEINDE'] == gemeinde].index,
        size=nachfrage_zentral)
    nachfragepunkte_höhere_Dichte = np.random.choice(
        gemeinden_höhere_Dichte_gdfs[gemeinden_höhere_Dichte_gdfs['GEMEINDE']
                                     == gemeinde].index, size=nachfrage_höhere_Dichte)
    nachfragepunkte_niedrige_Dichte = np.random.choice(
        gemeinden_niedrige_Dichte_gdfs[gemeinden_niedrige_Dichte_gdfs['GEMEINDE']
                                       == gemeinde].index, size=nachfrage_niedrige_Dichte)

    # Generiere Zeitstempel für die Nachfragepunkte
    nachfragepunkte_zentral_timestamps = generate_random_timestamp(start_timestamp, end_timestamp, nachfrage_zentral)
    nachfragepunkte_höhere_Dichte_timestamps = generate_random_timestamp(start_timestamp, end_timestamp, nachfrage_höhere_Dichte)
    nachfragepunkte_niedrige_Dichte_timestamps = generate_random_timestamp(start_timestamp, end_timestamp, nachfrage_niedrige_Dichte)

    return (nachfragepunkte_zentral, nachfragepunkte_zentral_timestamps), \
           (nachfragepunkte_höhere_Dichte, nachfragepunkte_höhere_Dichte_timestamps), \
           (nachfragepunkte_niedrige_Dichte, nachfragepunkte_niedrige_Dichte_timestamps)

def prepare_demand_data(gemeinden_zentral_gdfs, gemeinden_höhere_Dichte_gdfs, gemeinden_niedrige_Dichte_gdfs,
                        nachfrage_zentral, nachfrage_höhere_Dichte, nachfrage_niedrige_Dichte,
                        nachfrage_zentral_timestamps, nachfrage_höhere_Dichte_timestamps,
                        nachfrage_niedrige_Dichte_timestamps, Gemeinde):

    # Plotten der Nachfragepunkte für Meilen
    punkte_zentral = gemeinden_zentral_gdfs.loc[nachfrage_zentral, 'geometry'].centroid
    punkte_höhere_Dichte = gemeinden_höhere_Dichte_gdfs.loc[nachfrage_höhere_Dichte, 'geometry'].centroid
    punkte_niedrige_Dichte = gemeinden_niedrige_Dichte_gdfs.loc[nachfrage_niedrige_Dichte, 'geometry'].centroid

    # Erstellen eines GeoDataFrame für die Punkte von Meilen mit Zeitstempel
    punkte_zentral_gdf = gpd.GeoDataFrame(geometry=punkte_zentral, crs=gemeinden_zentral_gdfs.crs)
    punkte_zentral_gdf['timestamp'] = nachfrage_zentral_timestamps
    punkte_zentral_gdf['gemeinde'] = Gemeinde
    punkte_höhere_Dichte_gdf = gpd.GeoDataFrame(geometry=punkte_höhere_Dichte, crs=gemeinden_höhere_Dichte_gdfs.crs)
    punkte_höhere_Dichte_gdf['timestamp'] = nachfrage_höhere_Dichte_timestamps
    punkte_höhere_Dichte_gdf['gemeinde'] = Gemeinde
    punkte_niedrige_Dichte_gdf = gpd.GeoDataFrame(geometry=punkte_niedrige_Dichte, crs=gemeinden_niedrige_Dichte_gdfs.crs)
    punkte_niedrige_Dichte_gdf['timestamp'] = nachfrage_niedrige_Dichte_timestamps
    punkte_niedrige_Dichte_gdf['gemeinde'] = Gemeinde

    # Alle Punkte zusammenführen
    alle_punkte_gemeinde = pd.concat([punkte_zentral_gdf, punkte_höhere_Dichte_gdf, punkte_niedrige_Dichte_gdf])


    return alle_punkte_gemeinde

def add_passenger_numbers(gdf):
    gdf['passagier_nummer'] = range(1, len(gdf) + 1)
    return gdf
def xadd_destination_columns(input_gdf, gemeinden_zentral_gdfs, gemeinden_höhere_Dichte_gdfs,
                            gemeinden_niedrige_Dichte_gdfs):
    # Extrahiere die eindeutigen Gemeinden aus dem GeoDataFrame
    gemeinden = input_gdf['gemeinde'].unique()

    # Erstelle ein leeres DataFrame, um die neuen Zeilen hinzuzufügen
    new_rows = []

    # Iteriere über jede Gemeinde und zähle die Punkte
    for gemeinde in gemeinden:
        # Filtere das GeoDataFrame nach der aktuellen Gemeinde
        gemeinde_gdf = input_gdf[input_gdf['gemeinde'] == gemeinde]

        # Generiere die Zielpunkte für jeden Punkt in dieser Gemeinde
        for index, row in gemeinde_gdf.iterrows():
            # Erstelle eine Passagier-Nummer für jeden Punkt (aufsteigend)
            passagier_nummer = index + 1  # Aufsteigende Passagier-Nummer

            # Zufällige Auswahl einer anderen Gemeinde als Ziel
            ziel_gemeinde = np.random.choice([g for g in gemeinden if g != gemeinde])

            # Zufällige Auswahl eines Zielpunktes in der Zielgemeinde
            ziel_gemeinde_gdf = gemeinden_zentral_gdfs if ziel_gemeinde in gemeinden_zentral_gdfs['GEMEINDE'].values \
                else (gemeinden_höhere_Dichte_gdfs if ziel_gemeinde in gemeinden_höhere_Dichte_gdfs[
                'GEMEINDE'].values else gemeinden_niedrige_Dichte_gdfs)

            ziel_punkt_index = np.random.choice(ziel_gemeinde_gdf[ziel_gemeinde_gdf['GEMEINDE'] == ziel_gemeinde].index)

            # Füge den Zielpunkt als neue Zeile zum DataFrame hinzu
            ziel_punkt = ziel_gemeinde_gdf.loc[ziel_punkt_index, 'geometry'].centroid
            new_rows.append({'geometry': ziel_punkt,
                             'gemeinde': ziel_gemeinde,
                             'passagier_nummer': passagier_nummer})

    # Konvertiere die neuen Zeilen in ein DataFrame
    new_rows_df = pd.DataFrame(new_rows)

    # Füge die neuen Zeilen dem ursprünglichen GeoDataFrame hinzu
    input_gdf = pd.concat([input_gdf, new_rows_df], ignore_index=True)

    return input_gdf

def add_destination_columns(input_gdf, gemeinden_zentral_gdfs, gemeinden_höhere_Dichte_gdfs,
                            gemeinden_niedrige_Dichte_gdfs):
    # Extrahiere die eindeutigen Gemeinden aus dem GeoDataFrame
    gemeinden = input_gdf['gemeinde'].unique()

    # Erstelle ein leeres DataFrame, um die neuen Zeilen hinzuzufügen
    new_rows = []

    # Erstelle ein Dictionary, um das Kontingent für jede Zielgemeinde zu verfolgen
    kontingent_dict = {gemeinde: len(gemeinde_gdf) for gemeinde, gemeinde_gdf in input_gdf.groupby('gemeinde')}

    # Iteriere über jede Gemeinde und zähle die Punkte
    for gemeinde in gemeinden:
        # Filtere das GeoDataFrame nach der aktuellen Gemeinde
        gemeinde_gdf = input_gdf[input_gdf['gemeinde'] == gemeinde]

        # Generiere die Zielpunkte für jeden Punkt in dieser Gemeinde
        for index, row in gemeinde_gdf.iterrows():
            # Überprüfe, ob es noch Zielgemeinden mit nicht erschöpftem Kontingent gibt
            remaining_gemeinden = [g for g in gemeinden if g != gemeinde and kontingent_dict[g] > 0]
            if len(remaining_gemeinden) == 0:
                break  # Beende die Schleife, wenn alle Kontingente erschöpft sind

            # Wähle eine Zielgemeinde aus den verbleibenden aus
            ziel_gemeinde = np.random.choice(remaining_gemeinden)

            # Reduziere das Kontingent für die Zielgemeinde um 1
            kontingent_dict[ziel_gemeinde] -= 1

            # Zufällige Auswahl eines Zielpunktes in der Zielgemeinde
            ziel_gemeinde_gdf = gemeinden_zentral_gdfs if ziel_gemeinde in gemeinden_zentral_gdfs['GEMEINDE'].values \
                else (gemeinden_höhere_Dichte_gdfs if ziel_gemeinde in gemeinden_höhere_Dichte_gdfs[
                'GEMEINDE'].values else gemeinden_niedrige_Dichte_gdfs)

            ziel_punkt_index = np.random.choice(ziel_gemeinde_gdf[ziel_gemeinde_gdf['GEMEINDE'] == ziel_gemeinde].index)

            # Erstelle eine Passagier-Nummer für jeden Punkt (aufsteigend)
            passagier_nummer = len(new_rows) + 1  # Aufsteigende Passagier-Nummer

            # Füge den Zielpunkt als neue Zeile zum DataFrame hinzu
            ziel_punkt = ziel_gemeinde_gdf.loc[ziel_punkt_index, 'geometry'].centroid
            new_rows.append({'geometry': ziel_punkt,
                             'gemeinde': ziel_gemeinde,
                             'passagier_nummer': passagier_nummer})

    # Konvertiere die neuen Zeilen in ein DataFrame
    new_rows_df = pd.DataFrame(new_rows)

    # Füge die neuen Zeilen dem ursprünglichen GeoDataFrame hinzu
    input_gdf = pd.concat([input_gdf, new_rows_df], ignore_index=True)

    return input_gdf

import pandas as pd
import numpy as np
import geopandas as gpd

def create_destination_gdf(input_gdf, gemeinden_zentral_gdfs, gemeinden_höhere_Dichte_gdfs,
                            gemeinden_niedrige_Dichte_gdfs):
    # Extrahiere die eindeutigen Gemeinden aus dem GeoDataFrame
    gemeinden = input_gdf['gemeinde'].unique()

    # Erstelle ein leeres DataFrame, um die neuen Zeilen hinzuzufügen
    new_rows = []

    # Erstelle ein Dictionary, um das Kontingent für jede Zielgemeinde zu verfolgen
    kontingent_dict = {gemeinde: len(gemeinde_gdf) for gemeinde, gemeinde_gdf in input_gdf.groupby('gemeinde')}

    # Iteriere über jede Gemeinde und zähle die Punkte
    for gemeinde in gemeinden:
        # Filtere das GeoDataFrame nach der aktuellen Gemeinde
        gemeinde_gdf = input_gdf[input_gdf['gemeinde'] == gemeinde]

        # Generiere die Zielpunkte für jeden Punkt in dieser Gemeinde
        for index, row in gemeinde_gdf.iterrows():
            # Überprüfe, ob es noch Zielgemeinden mit nicht erschöpftem Kontingent gibt
            remaining_gemeinden = [g for g in gemeinden if g != gemeinde and kontingent_dict[g] >= 0]
            if len(remaining_gemeinden) == 0:
                break  # Beende die Schleife, wenn alle Kontingente erschöpft sind

            # Wähle eine Zielgemeinde aus den verbleibenden aus
            ziel_gemeinde = np.random.choice(remaining_gemeinden)

            # Reduziere das Kontingent für die Zielgemeinde um 1
            kontingent_dict[ziel_gemeinde] -= 1

            # Zufällige Auswahl eines Zielpunktes in der Zielgemeinde
            ziel_gemeinde_gdf = gemeinden_zentral_gdfs if ziel_gemeinde in gemeinden_zentral_gdfs['GEMEINDE'].values \
                else (gemeinden_höhere_Dichte_gdfs if ziel_gemeinde in gemeinden_höhere_Dichte_gdfs[
                'GEMEINDE'].values else gemeinden_niedrige_Dichte_gdfs)

            ziel_punkt_index = np.random.choice(ziel_gemeinde_gdf[ziel_gemeinde_gdf['GEMEINDE'] == ziel_gemeinde].index)

            # Erstelle eine Passagier-Nummer für jeden Punkt (aufsteigend)
            passagier_nummer = len(new_rows) + 1  # Aufsteigende Passagier-Nummer

            # Füge den Zielpunkt als neue Zeile zum DataFrame hinzu
            ziel_punkt = ziel_gemeinde_gdf.loc[ziel_punkt_index, 'geometry'].centroid
            new_rows.append({'geometry': ziel_punkt,
                             'gemeinde': ziel_gemeinde,
                             'passagier_nummer': passagier_nummer})

    # Konvertiere die neuen Zeilen in ein DataFrame
    new_rows_df = pd.DataFrame(new_rows)

    # Erstelle ein GeoDataFrame aus den neuen Zeilen
    destination_gdf = gpd.GeoDataFrame(new_rows_df, geometry='geometry')

    return destination_gdf




def filter_null_timestamp(gdf):
    filtered_gdf = gdf[gdf['timestamp'].isnull()]
    return filtered_gdf

import matplotlib.pyplot as plt
import geopandas as gpd
import osmnx as ox



# Liste der Farben für die Gemeinden
farben = {
    'Meilen': 'blue',
    'Egg': 'green',
    'Uster': 'red'
}
def plot_points(ax, punkt, farbe, label):
    ax.scatter(punkt.x, punkt.y, color=farbe, label=label)

def plot_demand_distribution(gemeindengrenzen_gdf, gemeinden_zentral_gdfs,
                             gemeinden_höhere_Dichte_gdfs, gemeinden_niedrige_Dichte_gdfs,
                             strassennetz_gdf, alle_punkte):

    fig, ax = plt.subplots(figsize=(10, 10))
    gemeindengrenzen_gdf.plot(ax=ax, color='black', edgecolor='black', alpha=0.5, label='Gemeindengrenzen')
    gemeinden_zentral_gdfs.plot(ax=ax, color='orange', edgecolor='gray', alpha=0.5, label='Zentral')
    gemeinden_höhere_Dichte_gdfs.plot(ax=ax, color='lightblue', edgecolor='gray', alpha=0.5, label='Höhere Dichte')
    gemeinden_niedrige_Dichte_gdfs.plot(ax=ax, color='lightgreen', edgecolor='gray', alpha=0.5, label='Niedrige Dichte')

    strassennetz_gdf.plot(ax=ax, color='gray', alpha=0.5, label='Straßennetz')
    plt.title('Verteilung der Nachfragepunkte in den Zonen')
    plt.xlabel('Longitude')
    plt.ylabel('Latitude')

    legend_labels = set()

    for index, row in alle_punkte.iterrows():
        punkt = row.geometry
        gemeinde = row.gemeinde
        if gemeinde not in legend_labels:
            plot_points(ax, punkt, farben[gemeinde], gemeinde)
            legend_labels.add(gemeinde)
        else:
            plot_points(ax, punkt, farben[gemeinde], None)

    ax.set_xlim([strassennetz_gdf.total_bounds[0], strassennetz_gdf.total_bounds[2]])
    ax.set_ylim([strassennetz_gdf.total_bounds[1], strassennetz_gdf.total_bounds[3]])
    ax.legend(title='Legende', loc='upper left')  # Legende außerhalb der Schleife platzieren

    plt.show()


def plot_routes(G, routes):
    fig, ax = ox.plot_graph_routes(G, routes, node_size=0)
    return fig, ax

In [2]:
import numpy as np
import geopandas as gpd
import pandas as pd
import matplotlib as plt
import os
#print(1)


# Erstelle Root zum Hauptordner
ROOT_FILES = 'C:/Users/Linus/PycharmProjects/BA/'
ROOT_RESOURCE = 'src/main/resources/QGIS/Nutzungsplan/'
ROOT_DOCS = 'src/main/resources/Dokumente/'
shapefile_paths = [ROOT_FILES + 'src/test/resources/input/Gebaut.shp', 'C:/Users/Linus/Documents/W1Gebaut.shp']
# Füge die verschiedene aus dem Inputornder Geo Files zusammen und speichere sie als gdf

gemeinden_gdfs = []
geo_files_tief = ['Gebaut.shp', 'W1Gebaut.shp', 'W30.gpkg', 'W2Uster.gpkg']
geo_files_hoch = ['W2Gebaut.gpkg', 'W2.gpkg', 'W3Uster.gpkg', 'W60.gpkg']
geo_files_zentral = ['Zentrum&Gewerbe.shp', 'ZonenOe.shp', 'Zentrum.gpkg', 'ZentrumUster.gpkg', 'ZentrumEgg.gpkg']

gdf_tiefe_Dichte = create_geo_dataframe(geo_files_tief)
gdf_hohe_Dichte = create_geo_dataframe(geo_files_hoch)
gdf_zentrale_Dichte = create_geo_dataframe(geo_files_zentral)

#print(gdf_tiefe_Dichte)
#Importiere die Datensätze für das Verkehrsmodell, das Strassenetz und der Gemeindengrenzen
ROOT_RESOURCE_VERKEHRSMODELL = 'src/main/resources/Verkehrsmodell/'
DATEN_VERKEHRSMODELL = pd.read_csv(ROOT_FILES + ROOT_RESOURCE_VERKEHRSMODELL + 'KTZH_00001982_00003903(1).csv')

ROOT_RESOURCE_GEMEINDENGRENZEN = 'src/main/resources/QGIS/Gemeindegrenzen/'
DATEN_GEMEINDENGRENZEN = gpd.read_file(ROOT_FILES + ROOT_RESOURCE_GEMEINDENGRENZEN + 'Gemeindegrenzen.shp')

ROOT_RESOURCE_STRASSENNETZ = 'src/main/resources/QGIS/Strassen/'
DATEN_STRASSENNETZ = gpd.read_file(ROOT_FILES + ROOT_RESOURCE_STRASSENNETZ + 'AlleStrassen.shp')

#Analyse des Verkehrsmodell in den Gemeinden
gewuenschte_gemeinden = ['Meilen', 'Egg', 'Uster', 'Uetikon am See']
gewuenschte_zielnamen = ['Meilen', 'Egg', 'Uster', 'Uetikon am See']
gewuenschte_kategorien = ['Verkehrsaufkommen']
gewuenschte_verkehrsmittel = ['oev']
gewuenschtes_jahr = 2018
#DATENSATZ_VERKEHR = DATEN_VERKEHRSMODELL

DATEN_FILTERED = filter_data(DATEN_VERKEHRSMODELL, gewuenschtes_jahr,
                             gewuenschte_gemeinden, gewuenschte_zielnamen,
                             gewuenschte_kategorien, gewuenschte_verkehrsmittel)
#print(DATEN_FILTERED)
# Filtern der Daten für jeden Zielort
daten_meilen = filter_target_data(DATEN_FILTERED, 'Meilen')
daten_egg = filter_target_data(DATEN_FILTERED, 'Egg')
daten_uster = filter_target_data(DATEN_FILTERED, 'Uster')
daten_uetikon = filter_target_data(DATEN_FILTERED, 'Uetikon am See')
#print(daten_meilen)
#print(daten_egg)
#print(daten_uetikon)


# Annahme: Vordefinierte Werte für stunden_verkehrstag und prozent_verteilung_hoch/mittel/niedrig
stunden_verkehrstag = 12
prozent_verteilung_hoch = 0.5
prozent_verteilung_mittel = 0.3
prozent_verteilung_niedrig = 0.2

Anzahl_Meilen_Egg_oev = filter_and_calculate_traffic_data(
    DATEN_VERKEHRSMODELL, 'Meilen', 'Egg', 'oev', 2018, 'Verkehrsaufkommen'
)
Anzahl_Meilen_Egg_miv = filter_and_calculate_traffic_data(
    DATEN_VERKEHRSMODELL, 'Meilen', 'Egg', 'miv', 2018, 'Verkehrsaufkommen'
)
Anzahl_Uetikon_Egg_oev = filter_and_calculate_traffic_data(
    DATEN_VERKEHRSMODELL, 'Uetikon am See', 'Egg', 'oev', 2018, 'Verkehrsaufkommen'
)
Anzahl_Uetikon_Egg_miv = filter_and_calculate_traffic_data(
    DATEN_VERKEHRSMODELL, 'Uetikon am See', 'Egg', 'miv', 2018, 'Verkehrsaufkommen'
)
Anzahl_oetwil_maennedorf_miv = filter_and_calculate_traffic_data(
    DATEN_VERKEHRSMODELL, 'Oetwil am See', 'Männedorf', 'miv', 2018, 'Verkehrsaufkommen'
)
Anzahl_oetwil_maennedorf_oev = filter_and_calculate_traffic_data(
    DATEN_VERKEHRSMODELL, 'Oetwil am See', 'Männedorf', 'oev', 2018, 'Verkehrsaufkommen'
)
modal_split_oev_Oetwil = filter_and_calculate_traffic_data(
    DATEN_VERKEHRSMODELL, 'Oetwil am See', 'Männedorf', 'oev', 2018, 'Modal Split'
)
modal_split_oev_oetwil_uster = filter_and_calculate_traffic_data(
    DATEN_VERKEHRSMODELL, 'Oetwil am See', 'Uster', 'oev', 2018, 'Modal Split'
)

#print('miv oet männe', Anzahl_oetwil_maennedorf_miv)
#print('oev oet männe', Anzahl_oetwil_maennedorf_oev)
#print('modal split oet männe', modal_split_oev_Oetwil)

# Berechne das neue Verkehrsaufkommen für die Gemeinden Meilen und Uetikon
neues_verkehrsaufkommen_oev_Meilen_Egg = berechne_verkehrsaufkommen(
    Anzahl_Meilen_Egg_oev, Anzahl_Meilen_Egg_miv, modal_split_oev_Oetwil) - Anzahl_Meilen_Egg_oev
#print('Meilen egg', neues_verkehrsaufkommen_oev_Meilen_Egg)
#print('Meilen egg oev alt', Anzahl_Meilen_Egg_oev)
#print('Meilen egg miv alt', Anzahl_Meilen_Egg_miv)

neues_verkehrsaufkommen_oev_Uetikon_Egg = berechne_verkehrsaufkommen(
    Anzahl_Uetikon_Egg_oev, Anzahl_Uetikon_Egg_miv, modal_split_oev_Oetwil) - Anzahl_Uetikon_Egg_oev
#print('Uetikon egg', neues_verkehrsaufkommen_oev_Uetikon_Egg)
#print('Uetikon egg oev alt', Anzahl_Uetikon_Egg_oev)
#print('Uetikon egg miv alt', Anzahl_Uetikon_Egg_miv)



nachfrage_meilen, nachfrage_meilen_zentral,\
    nachfrage_meilen_höhere_Dichte, nachfrage_meilen_niedrige_Dichte =\
    calculate_new_demand(daten_meilen, neues_verkehrsaufkommen_oev_Meilen_Egg, stunden_verkehrstag,
                     prozent_verteilung_hoch, prozent_verteilung_mittel,
                     prozent_verteilung_niedrig)
nachfrage_egg, nachfrage_egg_zentral,\
    nachfrage_egg_höhere_Dichte, nachfrage_egg_niedrige_Dichte =\
    calculate_new_demand(daten_egg, neues_verkehrsaufkommen_oev_Meilen_Egg + neues_verkehrsaufkommen_oev_Uetikon_Egg, stunden_verkehrstag,
                     prozent_verteilung_hoch, prozent_verteilung_mittel,
                     prozent_verteilung_niedrig)
nachfrage_uster, nachfrage_uster_zentral,\
    nachfrage_uster_höhere_Dichte, nachfrage_uster_niedrige_Dichte =\
    calculate_demand(daten_uster, stunden_verkehrstag,
                     prozent_verteilung_hoch, prozent_verteilung_mittel,
                     prozent_verteilung_niedrig)

nachfrage_uetikon, nachfrage_uetikon_zentral,\
    nachfrage_uetikon_höhere_Dichte, nachfrage_uetikon_niedrige_Dichte =\
    calculate_new_demand(daten_uetikon, neues_verkehrsaufkommen_oev_Uetikon_Egg, stunden_verkehrstag,
                     prozent_verteilung_hoch, prozent_verteilung_mittel,
                     prozent_verteilung_niedrig)

# Setze die Zeitstempel für die Nachfragepunkte
start_timestamp = pd.Timestamp('2018-04-20 09:00:00')
end_timestamp = pd.Timestamp('2018-04-20 10:00:00')

# Verteile die Nachfragepunkte in den Gemeinden
(nachfrage_meilen_zentral_pos, nachfrage_meilen_zentral_timestamps), \
(nachfrage_meilen_höhere_Dichte_pos, nachfrage_meilen_höhere_Dichte_timestamps), \
(nachfrage_meilen_niedrige_Dichte_pos, nachfrage_meilen_niedrige_Dichte_timestamps) = \
    verteile_nachfragepunkte('Meilen', nachfrage_meilen_zentral,
                             nachfrage_meilen_höhere_Dichte,
                             nachfrage_meilen_niedrige_Dichte,
                             gdf_zentrale_Dichte, gdf_hohe_Dichte,
                             gdf_tiefe_Dichte, start_timestamp, end_timestamp)


(nachfrage_egg_zentral_pos, nachfrage_egg_zentral_timestamps), \
(nachfrage_egg_höhere_Dichte_pos, nachfrage_egg_höhere_Dichte_timestamps), \
(nachfrage_egg_niedrige_Dichte_pos, nachfrage_egg_niedrige_Dichte_timestamps) = \
    verteile_nachfragepunkte('Egg', nachfrage_egg_zentral,
                             nachfrage_egg_höhere_Dichte,
                             nachfrage_egg_niedrige_Dichte,
                             gdf_zentrale_Dichte, gdf_hohe_Dichte,
                             gdf_tiefe_Dichte, start_timestamp, end_timestamp)

(nachfrage_uster_zentral_pos, nachfrage_uster_zentral_timestamps), \
(nachfrage_uster_höhere_Dichte_pos, nachfrage_uster_höhere_Dichte_timestamps), \
(nachfrage_uster_niedrige_Dichte_pos, nachfrage_uster_niedrige_Dichte_timestamps) = \
    verteile_nachfragepunkte('Uster', nachfrage_uster_zentral,
                             nachfrage_uster_höhere_Dichte,
                             nachfrage_uster_niedrige_Dichte,
                             gdf_zentrale_Dichte, gdf_hohe_Dichte,
                             gdf_tiefe_Dichte, start_timestamp, end_timestamp)

(nachfrage_uetikon_zentral_pos, nachfrage_uetikon_zentral_timestamps), \
(nachfrage_uetikon_höhere_Dichte_pos, nachfrage_uetikon_höhere_Dichte_timestamps), \
(nachfrage_uetikon_niedrige_Dichte_pos, nachfrage_uetikon_niedrige_Dichte_timestamps) = \
    verteile_nachfragepunkte('Uetikon am See', nachfrage_uetikon_zentral,
                                nachfrage_uetikon_höhere_Dichte,
                                nachfrage_uetikon_niedrige_Dichte,
                                gdf_zentrale_Dichte, gdf_hohe_Dichte,
                                gdf_tiefe_Dichte, start_timestamp, end_timestamp)

#print(nachfrage_meilen_zentral)
#print('Daten Meilen:' , nachfrage_meilen)
#print('Daten Meilen:' , nachfrage_meilen_zentral_pos)
#print('Daten Meilen:' , nachfrage_meilen_höhere_Dichte_pos)
#print('Daten Meilen:' , nachfrage_meilen_niedrige_Dichte_pos)
#print('Daten Egg:' , nachfrage_egg)
#print('Daten Egg:' , nachfrage_egg_zentral_pos)
print('Daten Uster:' , nachfrage_uster)
#print('Daten Uster:' , nachfrage_uster_zentral_pos)
print('Daten Uster:' , nachfrage_uster_höhere_Dichte_pos)
print('Daten Uster:' , nachfrage_uster_niedrige_Dichte_timestamps)
print('Daten Uetikon:' , nachfrage_uetikon)
#print('Daten Uetikon:' , nachfrage_uetikon_zentral_pos)
#print('Daten Uetikon:' , nachfrage_uetikon_höhere_Dichte_pos)
#print('Daten Uetikon:' , nachfrage_uetikon_niedrige_Dichte_pos)

# Rufe die Funktion auf, um die Daten vorzubereiten
meilen_punkte_df = prepare_demand_data(gdf_zentrale_Dichte, gdf_hohe_Dichte, gdf_tiefe_Dichte,
                                       nachfrage_meilen_zentral_pos, nachfrage_meilen_höhere_Dichte_pos,
                                       nachfrage_meilen_niedrige_Dichte_pos, nachfrage_meilen_zentral_timestamps,
                                       nachfrage_meilen_höhere_Dichte_timestamps, nachfrage_meilen_niedrige_Dichte_timestamps,
                                       'Meilen')
egg_punkte_df = prepare_demand_data(gdf_zentrale_Dichte, gdf_hohe_Dichte, gdf_tiefe_Dichte,
                                       nachfrage_egg_zentral_pos, nachfrage_egg_höhere_Dichte_pos,
                                       nachfrage_egg_niedrige_Dichte_pos, nachfrage_egg_zentral_timestamps,
                                       nachfrage_egg_höhere_Dichte_timestamps, nachfrage_egg_niedrige_Dichte_timestamps,
                                       'Egg')

uster_punkte_df = prepare_demand_data(gdf_zentrale_Dichte, gdf_hohe_Dichte, gdf_tiefe_Dichte,
                                       nachfrage_uster_zentral_pos, nachfrage_uster_höhere_Dichte_pos,
                                       nachfrage_uster_niedrige_Dichte_pos, nachfrage_uster_zentral_timestamps,
                                       nachfrage_uster_höhere_Dichte_timestamps, nachfrage_uster_niedrige_Dichte_timestamps,
                                      'Uster')

uetikon_punkte_df = prepare_demand_data(gdf_zentrale_Dichte, gdf_hohe_Dichte, gdf_tiefe_Dichte,
                                       nachfrage_uetikon_zentral_pos, nachfrage_uetikon_höhere_Dichte_pos,
                                       nachfrage_uetikon_niedrige_Dichte_pos, nachfrage_uetikon_zentral_timestamps,
                                       nachfrage_uetikon_höhere_Dichte_timestamps, nachfrage_uetikon_niedrige_Dichte_timestamps,
                                       'Uetikon am See')

print('Meilen:', meilen_punkte_df)
# Verwendung von aussagekräftigen Variablennamen und Inline-Kommentaren
alle_punkte_df = pd.concat([egg_punkte_df, uster_punkte_df, meilen_punkte_df, uetikon_punkte_df], ignore_index=True)
alle_punkte_df_with_passenger_numbers = add_passenger_numbers(alle_punkte_df)

alle_punkte_df_with_passenger_numbers.to_file(ROOT_FILES + ROOT_DOCS + "Nachfrage.geojson", driver='GeoJSON')

# Plotten der Nachfrageverteilung
#plot_demand_distribution(DATEN_GEMEINDENGRENZEN, gdf_zentrale_Dichte,
                         #gdf_hohe_Dichte, gdf_tiefe_Dichte,
                         #DATEN_STRASSENNETZ, alle_punkte_df_with_passenger_numbers)
# Beispielaufruf:
new_destination_gdf = create_destination_gdf(alle_punkte_df_with_passenger_numbers, gdf_zentrale_Dichte, gdf_hohe_Dichte,
                                             gdf_tiefe_Dichte)
# Speichern der GeoDataFrame als GeoJSON-Datei
new_destination_gdf.to_file(ROOT_FILES + ROOT_DOCS + "Ziele.geojson", driver='GeoJSON')

# Anwenden der Funktion für das Hinzufügen von Zielspalten
output_gdf = add_destination_columns(alle_punkte_df_with_passenger_numbers, gdf_zentrale_Dichte,
                                      gdf_hohe_Dichte, gdf_tiefe_Dichte)

# Speichern der GeoDataFrame als GeoJSON-Datei
output_gdf.to_file(ROOT_FILES + ROOT_DOCS + "output.geojson", driver='GeoJSON')

destination_gdf = filter_null_timestamp(output_gdf)

#plot_demand_distribution(DATEN_GEMEINDENGRENZEN, gdf_zentrale_Dichte,
                         #gdf_hohe_Dichte, gdf_tiefe_Dichte,
                         #DATEN_STRASSENNETZ, destination_gdf)


Daten Uster: 20
Daten Uster: [117 128  82  82 118  91]
Daten Uster: [Timestamp('2018-04-20 09:08:10'), Timestamp('2018-04-20 09:43:01'), Timestamp('2018-04-20 09:35:21'), Timestamp('2018-04-20 09:57:00')]
Daten Uetikon: 23
Meilen:                            geometry           timestamp gemeinde
34  POINT (2692215.684 1235902.103) 2018-04-20 09:19:37   Meilen
39  POINT (2690856.548 1236326.424) 2018-04-20 09:12:02   Meilen
4   POINT (2690820.691 1236119.105) 2018-04-20 09:14:54   Meilen
43  POINT (2689307.510 1237515.137) 2018-04-20 09:36:45   Meilen
28  POINT (2691108.391 1236294.995) 2018-04-20 09:28:25   Meilen
33  POINT (2691869.011 1235914.026) 2018-04-20 09:54:39   Meilen
19  POINT (2689441.545 1236879.293) 2018-04-20 09:28:07   Meilen
36  POINT (2691391.077 1236161.705) 2018-04-20 09:08:51   Meilen
48  POINT (2691463.495 1236544.313) 2018-04-20 09:47:36   Meilen
15  POINT (2692350.320 1235639.143) 2018-04-20 09:41:39   Meilen
20  POINT (2689839.988 1236755.053) 2018-04-20 09:39:0

In [3]:
def calculate_demand_bahnhof(daten, daten_bahnhof, stunden_verkehrstag, prozent_verteilung_hoch,
                     prozent_verteilung_mittel, prozent_verteilung_niedrig):
    nachfrage = int(daten['wert'].sum() / stunden_verkehrstag)
    nachfrage_bahnhof = int(((daten_bahnhof['wert'].sum()) / stunden_verkehrstag) / 2)
    nachfrage_gemeinde = nachfrage + nachfrage_bahnhof
    #nachfrage_zentral = int(prozent_verteilung_hoch * nachfrage)
    nachfrage_höhere_Dichte = int(prozent_verteilung_mittel * nachfrage_gemeinde)
    nachfrage_niedrige_Dichte = int(prozent_verteilung_niedrig * nachfrage_gemeinde)
    nachfrage_zentral = nachfrage_gemeinde - nachfrage_höhere_Dichte - nachfrage_niedrige_Dichte
    return nachfrage_gemeinde, nachfrage_bahnhof, nachfrage_zentral,\
        nachfrage_höhere_Dichte, nachfrage_niedrige_Dichte

def verteile_nachfragepunkte_bahnhof(gemeinde, nachfrage_zentral, nachfrage_höhere_Dichte,
                             nachfrage_niedrige_Dichte, nachfrage_bahnhof, gemeinden_zentral_gdfs,
                             gemeinden_höhere_Dichte_gdfs, gemeinden_niedrige_Dichte_gdfs,
                             start_timestamp, end_timestamp):
    nachfragepunkte_zentral = np.random.choice(
        gemeinden_zentral_gdfs[gemeinden_zentral_gdfs['GEMEINDE'] == gemeinde].index,
        size=nachfrage_zentral)
    nachfragepunkte_höhere_Dichte = np.random.choice(
        gemeinden_höhere_Dichte_gdfs[gemeinden_höhere_Dichte_gdfs['GEMEINDE']
                                     == gemeinde].index, size=nachfrage_höhere_Dichte)
    nachfragepunkte_niedrige_Dichte = np.random.choice(
        gemeinden_niedrige_Dichte_gdfs[gemeinden_niedrige_Dichte_gdfs['GEMEINDE']
                                       == gemeinde].index, size=nachfrage_niedrige_Dichte)

    # Generiere Zeitstempel für die Nachfragepunkte
    nachfragepunkte_zentral_timestamps = generate_random_timestamp(start_timestamp, end_timestamp, nachfrage_zentral)
    nachfragepunkte_höhere_Dichte_timestamps = generate_random_timestamp(start_timestamp, end_timestamp, nachfrage_höhere_Dichte)
    nachfragepunkte_niedrige_Dichte_timestamps = generate_random_timestamp(start_timestamp, end_timestamp, nachfrage_niedrige_Dichte)
    nachfragepunkte_bahnhof_timestamps = generate_random_timestamp(start_timestamp, end_timestamp, nachfrage_bahnhof)

    return (nachfragepunkte_zentral, nachfragepunkte_zentral_timestamps), \
           (nachfragepunkte_höhere_Dichte, nachfragepunkte_höhere_Dichte_timestamps), \
           (nachfragepunkte_niedrige_Dichte, nachfragepunkte_niedrige_Dichte_timestamps), \
           nachfragepunkte_bahnhof_timestamps

def prepare_demand_data_bahnhof(gemeinden_zentral_gdfs, gemeinden_höhere_Dichte_gdfs, gemeinden_niedrige_Dichte_gdfs,
                        nachfrage_zentral, nachfrage_höhere_Dichte, nachfrage_niedrige_Dichte,
                        nachfrage_zentral_timestamps, nachfrage_höhere_Dichte_timestamps,
                        nachfrage_niedrige_Dichte_timestamps, Gemeinde, bahnhof_gdf, nachfrage_bahnhof_timestamps):

    # Plotten der Nachfragepunkte für Meilen
    punkte_zentral = gemeinden_zentral_gdfs.loc[nachfrage_zentral, 'geometry'].centroid
    punkte_höhere_Dichte = gemeinden_höhere_Dichte_gdfs.loc[nachfrage_höhere_Dichte, 'geometry'].centroid
    punkte_niedrige_Dichte = gemeinden_niedrige_Dichte_gdfs.loc[nachfrage_niedrige_Dichte, 'geometry'].centroid

    # Erstellen eines GeoDataFrame für die Punkte von Meilen mit Zeitstempel
    punkte_zentral_gdf = gpd.GeoDataFrame(geometry=punkte_zentral, crs=gemeinden_zentral_gdfs.crs)
    punkte_zentral_gdf['timestamp'] = nachfrage_zentral_timestamps
    punkte_zentral_gdf['gemeinde'] = Gemeinde
    punkte_höhere_Dichte_gdf = gpd.GeoDataFrame(geometry=punkte_höhere_Dichte, crs=gemeinden_höhere_Dichte_gdfs.crs)
    punkte_höhere_Dichte_gdf['timestamp'] = nachfrage_höhere_Dichte_timestamps
    punkte_höhere_Dichte_gdf['gemeinde'] = Gemeinde
    punkte_niedrige_Dichte_gdf = gpd.GeoDataFrame(geometry=punkte_niedrige_Dichte, crs=gemeinden_niedrige_Dichte_gdfs.crs)
    punkte_niedrige_Dichte_gdf['timestamp'] = nachfrage_niedrige_Dichte_timestamps
    punkte_niedrige_Dichte_gdf['gemeinde'] = Gemeinde

    # Extrahieren der Geometrie basierend auf dem Namen
    geometry = bahnhof_gdf.loc[bahnhof_gdf['name'] == Gemeinde, 'geometry'].values[0]

    # Erstellen eines neuen DataFrame mit den Timestamps und der extrahierten Geometrie
    punkte_bahnhof = pd.DataFrame({
        'timestamp': nachfrage_bahnhof_timestamps,
        'geometry': [geometry] * len(nachfrage_bahnhof_timestamps)
    })
    punkte_bahnhof_gdf = gpd.GeoDataFrame(punkte_bahnhof, geometry='geometry', crs=bahnhof_gdf.crs)
    punkte_bahnhof_gdf['timestamp'] = nachfrage_bahnhof_timestamps
    punkte_bahnhof_gdf['gemeinde'] = Gemeinde

    # Alle Punkte zusammenführen
    alle_punkte_gemeinde = pd.concat([punkte_zentral_gdf, punkte_höhere_Dichte_gdf, punkte_niedrige_Dichte_gdf, punkte_bahnhof_gdf])

    return alle_punkte_gemeinde

import numpy as np
import pandas as pd
import geopandas as gpd

def create_destination_zuglinie_gdf(input_gdf, gemeinden_zentral_gdfs, gemeinden_höhere_Dichte_gdfs,
                            gemeinden_niedrige_Dichte_gdfs, bahnhöfe_gdf):
    # Extrahiere die eindeutigen Gemeinden aus dem GeoDataFrame
    gemeinden = input_gdf['gemeinde'].unique()

    # Erstelle ein leeres DataFrame, um die neuen Zeilen hinzuzufügen
    new_rows = []

    # Erstelle ein Dictionary, um das Kontingent für jede Zielgemeinde zu verfolgen
    kontingent_dict = {gemeinde: len(gemeinde_gdf) for gemeinde, gemeinde_gdf in input_gdf.groupby('gemeinde')}

    # Erstelle ein Dictionary, um das Kontingent für die Bahnhöfe zu verfolgen
    bahnhof_kontingent_dict = {row['geometry']: len(input_gdf[input_gdf['geometry'] == row['geometry']]) for _, row in bahnhöfe_gdf.iterrows()}

    # Iteriere über jede Gemeinde und zähle die Punkte
    for gemeinde in gemeinden:
        # Filtere das GeoDataFrame nach der aktuellen Gemeinde
        gemeinde_gdf = input_gdf[input_gdf['gemeinde'] == gemeinde]

        # Generiere die Zielpunkte für jeden Punkt in dieser Gemeinde
        for index, row in gemeinde_gdf.iterrows():
            # Überprüfe, ob der aktuelle Punkt ein Bahnhofspunkt ist
            if row['geometry'] in bahnhof_kontingent_dict and bahnhof_kontingent_dict[row['geometry']] > 0:
                # Zufällig entscheiden, ob der Punkt auf dem Bahnhof oder in der Zone verteilt wird
                if np.random.rand() < 0.5:
                    # Punkt bleibt auf dem Bahnhof
                    ziel_punkt = row['geometry']
                else:
                    # Punkt wird zufällig in den Zonen verteilt
                    remaining_gemeinden = [g for g in gemeinden if g != gemeinde and kontingent_dict[g] >= 0]
                    if len(remaining_gemeinden) > 0:
                        ziel_gemeinde = np.random.choice(remaining_gemeinden)
                        kontingent_dict[ziel_gemeinde] -= 1
                        
                        ziel_gemeinde_gdf = gemeinden_zentral_gdfs if ziel_gemeinde in gemeinden_zentral_gdfs['GEMEINDE'].values \
                            else (gemeinden_höhere_Dichte_gdfs if ziel_gemeinde in gemeinden_höhere_Dichte_gdfs['GEMEINDE'].values else gemeinden_niedrige_Dichte_gdfs)
                        
                        ziel_punkt_index = np.random.choice(ziel_gemeinde_gdf[ziel_gemeinde_gdf['GEMEINDE'] == ziel_gemeinde].index)
                        ziel_punkt = ziel_gemeinde_gdf.loc[ziel_punkt_index, 'geometry'].centroid
                    else:
                        ziel_punkt = row['geometry']
                
                bahnhof_kontingent_dict[row['geometry']] -= 1
            else:
                # Überprüfe, ob es noch Zielgemeinden mit nicht erschöpftem Kontingent gibt
                remaining_gemeinden = [g for g in gemeinden if g != gemeinde and kontingent_dict[g] >= 0]
                if len(remaining_gemeinden) == 0:
                    break  # Beende die Schleife, wenn alle Kontingente erschöpft sind

                # Wähle eine Zielgemeinde aus den verbleibenden aus
                ziel_gemeinde = np.random.choice(remaining_gemeinden)

                # Reduziere das Kontingent für die Zielgemeinde um 1
                kontingent_dict[ziel_gemeinde] -= 1

                # Zufällige Auswahl eines Zielpunktes in der Zielgemeinde
                ziel_gemeinde_gdf = gemeinden_zentral_gdfs if ziel_gemeinde in gemeinden_zentral_gdfs['GEMEINDE'].values \
                    else (gemeinden_höhere_Dichte_gdfs if ziel_gemeinde in gemeinden_höhere_Dichte_gdfs['GEMEINDE'].values else gemeinden_niedrige_Dichte_gdfs)

                ziel_punkt_index = np.random.choice(ziel_gemeinde_gdf[ziel_gemeinde_gdf['GEMEINDE'] == ziel_gemeinde].index)
                ziel_punkt = ziel_gemeinde_gdf.loc[ziel_punkt_index, 'geometry'].centroid

            # Erstelle eine Passagier-Nummer für jeden Punkt (aufsteigend)
            passagier_nummer = len(new_rows) + 1  # Aufsteigende Passagier-Nummer

            # Füge den Zielpunkt als neue Zeile zum DataFrame hinzu
            new_rows.append({'geometry': ziel_punkt,
                             'gemeinde': ziel_gemeinde if row['geometry'] not in bahnhof_kontingent_dict else gemeinde,
                             'passagier_nummer': passagier_nummer})

    # Konvertiere die neuen Zeilen in ein DataFrame
    new_rows_df = pd.DataFrame(new_rows)

    # Erstelle ein GeoDataFrame aus den neuen Zeilen
    destination_gdf = gpd.GeoDataFrame(new_rows_df, geometry='geometry')

    return destination_gdf


In [4]:
ROOT_FILES = 'C:/Users/Linus/PycharmProjects/BA/'
ROOT_ODPT_stops = 'src/main/resources/ODPT/'

#Pfad zur Shapefile-Datei mit den Strassen
ROOT_RESOURCE_STRASSENNETZ = 'src/main/resources/QGIS/Strassen/'
#DATEN_STRASSENNETZ = gpd.read_file(ROOT_FILES + ROOT_RESOURCE_STRASSENNETZ + 'Tramstrasse.shp')

# save graph as a geopackage or graphml file
#ox.io.save_graph_geopackage(G, filepath = ROOT_FILES + ROOT_RESOURCE_STRASSENNETZ + "./OSMStrassennetz.gpkg")

# Pfad zur Shapefile-Datei mit den Bushaltestellen
shapefile_path = ROOT_FILES + ROOT_ODPT_stops + "ODPTSTOPS.shp"

# Laden der Bushaltestellen als GeoDataFrame
ODPT_stops = gpd.read_file(shapefile_path)

# Erstellen der Nachfragepunkte zu den Bahnhöfen
gemeinden_zuglinie_Meilen = ['Meilen', 'Zürich', 'Zollikon', 'Küsnacht (ZH)', 'Erlenbach (ZH)', 'Herrliberg', 'Uetikon am See', 'Männedorf', 'Stäfa', 'Hombrechtikon']
zielgemeinden_zuglinie_meilen = ['Meilen', 'Zürich', 'Zollikon', 'Küsnacht (ZH)', 'Erlenbach (ZH)', 'Herrliberg', 'Uetikon am See', 'Männedorf', 'Stäfa', 'Hombrechtikon']
gemeinden_zuglinie_egg = ['Egg', 'Zürich', 'Zollikon', 'Küsnacht (ZH)', 'Maur', 'Zumikon']
zielgemeinden_zuglinie_egg = ['Egg', 'Zürich', 'Zollikon', 'Küsnacht (ZH)', 'Maur', 'Zumikon']
gemeinden_zuglinie_uster = ['Uster', 'Zürich', 'Dübendorf', 'Greifensee', 'Schwerzenbach', 'Seegräben', 'Wetzikon', 'Bubikon', 'Rüti (ZH)']
zielgemeinden_zuglinie_uster = ['Uster', 'Zürich', 'Dübendorf', 'Greifensee', 'Schwerzenbach', 'Seegräben', 'Wetzikon', 'Bubikon', 'Rüti (ZH)']
gewuenschte_kategorien_zuglinie = ['Verkehrsaufkommen']
gewuenschte_verkehrsmittel_zuglinie = ['oev']
gewuenschtes_jahr_zuglinie = 2018

DATEN_FILTERED_zuglinie_meilen = filter_data(DATEN_VERKEHRSMODELL, gewuenschtes_jahr_zuglinie,
                             gemeinden_zuglinie_Meilen, zielgemeinden_zuglinie_meilen,
                             gewuenschte_kategorien_zuglinie, gewuenschte_verkehrsmittel_zuglinie)
#print(DATEN_FILTERED_zuglinie_meilen)
daten_meilen_zuglinie = filter_target_data(DATEN_FILTERED_zuglinie_meilen, 'Meilen')
#print(daten_meilen_zuglinie)

DATEN_FILTERED_zuglinie_egg = filter_data(DATEN_VERKEHRSMODELL, gewuenschtes_jahr_zuglinie,
                                gemeinden_zuglinie_egg, zielgemeinden_zuglinie_egg,
                                gewuenschte_kategorien_zuglinie, gewuenschte_verkehrsmittel_zuglinie)
#print(DATEN_FILTERED_zuglinie_egg)
daten_egg_zuglinie = filter_target_data(DATEN_FILTERED_zuglinie_egg, 'Egg')

DATEN_FILTERED_zuglinie_uster = filter_data(DATEN_VERKEHRSMODELL, gewuenschtes_jahr_zuglinie,
                                gemeinden_zuglinie_uster, zielgemeinden_zuglinie_uster,
                                gewuenschte_kategorien_zuglinie, gewuenschte_verkehrsmittel_zuglinie)
#print(DATEN_FILTERED_zuglinie_uster)
daten_uster_zuglinie = filter_target_data(DATEN_FILTERED_zuglinie_uster, 'Uster')


nachfrage_meilen_zuglinie, nachfrage_meilen_zuglinie_bahnhof, \
    nachfrage_meilen_zentral, nachfrage_meilen_höhere_Dichte, nachfrage_meilen_niedrige_Dichte \
            = calculate_demand_bahnhof(daten_meilen, daten_meilen_zuglinie, stunden_verkehrstag, \
                                       prozent_verteilung_hoch, prozent_verteilung_mittel, prozent_verteilung_niedrig)

nachfrage_egg_zuglinie, nachfrage_egg_zuglinie_bahnhof, \
    nachfrage_egg_zentral, nachfrage_egg_höhere_Dichte, nachfrage_egg_niedrige_Dichte \
            = calculate_demand_bahnhof(daten_egg, daten_egg_zuglinie, stunden_verkehrstag, \
                                        prozent_verteilung_hoch, prozent_verteilung_mittel, prozent_verteilung_niedrig)

nachfrage_uster_zuglinie, nachfrage_uster_zuglinie_bahnhof, \
    nachfrage_uster_zentral, nachfrage_uster_höhere_Dichte, nachfrage_uster_niedrige_Dichte \
            = calculate_demand_bahnhof(daten_uster, daten_uster_zuglinie, stunden_verkehrstag, \
                                        prozent_verteilung_hoch, prozent_verteilung_mittel, prozent_verteilung_niedrig)



(nachfrage_meilen_zentral_pos, nachfrage_meilen_zentral_timestamps), \
(nachfrage_meilen_höhere_Dichte_pos, nachfrage_meilen_höhere_Dichte_timestamps), \
(nachfrage_meilen_niedrige_Dichte_pos, nachfrage_meilen_niedrige_Dichte_timestamps), \
    nachfragepunkte_bahnhof_timestamps  = \
    verteile_nachfragepunkte('Meilen', nachfrage_meilen_zentral,
                             nachfrage_meilen_höhere_Dichte,
                             nachfrage_meilen_niedrige_Dichte,
                             nachfrage_meilen_zuglinie_bahnhof,
                             gdf_zentrale_Dichte, gdf_hohe_Dichte,
                             gdf_tiefe_Dichte, start_timestamp, end_timestamp)

(nachfrage_egg_zentral_pos, nachfrage_egg_zentral_timestamps), \
(nachfrage_egg_höhere_Dichte_pos, nachfrage_egg_höhere_Dichte_timestamps), \
(nachfrage_egg_niedrige_Dichte_pos, nachfrage_egg_niedrige_Dichte_timestamps), \
    nachfragepunkte_bahnhof_timestamps  = \
    verteile_nachfragepunkte('Egg', nachfrage_egg_zentral,
                                nachfrage_egg_höhere_Dichte,
                                nachfrage_egg_niedrige_Dichte,
                                nachfrage_egg_zuglinie_bahnhof,
                                gdf_zentrale_Dichte, gdf_hohe_Dichte,
                                gdf_tiefe_Dichte, start_timestamp, end_timestamp)

(nachfrage_uster_zentral_pos, nachfrage_uster_zentral_timestamps), \
(nachfrage_uster_höhere_Dichte_pos, nachfrage_uster_höhere_Dichte_timestamps), \
(nachfrage_uster_niedrige_Dichte_pos, nachfrage_uster_niedrige_Dichte_timestamps), \
    nachfragepunkte_bahnhof_timestamps  = \
    verteile_nachfragepunkte('Uster', nachfrage_uster_zentral,
                                nachfrage_uster_höhere_Dichte,
                                nachfrage_uster_niedrige_Dichte,
                                nachfrage_uster_zuglinie_bahnhof,
                                gdf_zentrale_Dichte, gdf_hohe_Dichte,
                                gdf_tiefe_Dichte, start_timestamp, end_timestamp)

meilen_punkte_df = prepare_demand_data(gdf_zentrale_Dichte, gdf_hohe_Dichte, gdf_tiefe_Dichte,
                                       nachfrage_meilen_zentral_pos, nachfrage_meilen_höhere_Dichte_pos,
                                       nachfrage_meilen_niedrige_Dichte_pos, nachfrage_meilen_zentral_timestamps,
                                       nachfrage_meilen_höhere_Dichte_timestamps, nachfrage_meilen_niedrige_Dichte_timestamps,
                                       'Meilen', ODPT_stops, nachfragepunkte_bahnhof_timestamps)

egg_punkte_df = prepare_demand_data(gdf_zentrale_Dichte, gdf_hohe_Dichte, gdf_tiefe_Dichte,
                                    nachfrage_egg_zentral_pos, nachfrage_egg_höhere_Dichte_pos,
                                    nachfrage_egg_niedrige_Dichte_pos, nachfrage_egg_zentral_timestamps,
                                    nachfrage_egg_höhere_Dichte_timestamps, nachfrage_egg_niedrige_Dichte_timestamps,
                                    'Egg', ODPT_stops, nachfragepunkte_bahnhof_timestamps)

uster_punkte_df = prepare_demand_data(gdf_zentrale_Dichte, gdf_hohe_Dichte, gdf_tiefe_Dichte,
                                        nachfrage_uster_zentral_pos, nachfrage_uster_höhere_Dichte_pos,
                                        nachfrage_uster_niedrige_Dichte_pos, nachfrage_uster_zentral_timestamps,
                                        nachfrage_uster_höhere_Dichte_timestamps, nachfrage_uster_niedrige_Dichte_timestamps,
                                        'Uster', ODPT_stops, nachfragepunkte_bahnhof_timestamps)

alle_punkte_zuglinie_df = pd.concat([egg_punkte_df, uster_punkte_df, meilen_punkte_df], ignore_index=True)
alle_punkte_df_with_passenger_numbers_zuglinie = add_passenger_numbers(alle_punkte_zuglinie_df)

alle_punkte_df_with_passenger_numbers_zuglinie.to_file(ROOT_FILES + ROOT_DOCS + "Nachfrage_mit_zuglinie.geojson", driver='GeoJSON')


new_destination_zuglinie_gdf = create_destination_zuglinie_gdf(alle_punkte_df_with_passenger_numbers_zuglinie, gdf_zentrale_Dichte, gdf_hohe_Dichte,
                                             gdf_tiefe_Dichte, ODPT_stops)
# Speichern der GeoDataFrame als GeoJSON-Datei
new_destination_zuglinie_gdf.to_file(ROOT_FILES + ROOT_DOCS + "Ziele_mit_zuglinie.geojson", driver='GeoJSON')

# Anwenden der Funktion für das Hinzufügen von Zielspalten
output_zuglinie_gdf = add_destination_columns(alle_punkte_df_with_passenger_numbers, gdf_zentrale_Dichte,
                                      gdf_hohe_Dichte, gdf_tiefe_Dichte)

# Speichern der GeoDataFrame als GeoJSON-Datei
output_zuglinie_gdf.to_file(ROOT_FILES + ROOT_DOCS + "output_mit_zuglinie.geojson", driver='GeoJSON')

achse 23
bahnlinie 189
gemeinde 212
achse 17
bahnlinie 54
gemeinde 71
achse 20
bahnlinie 250
gemeinde 270


In [5]:
print(ODPT_stops)

   id    name                         geometry
0   1  Meilen  POINT (2691216.542 1236096.946)
1   2     Egg  POINT (2694601.091 1239702.123)
2   3   Uster  POINT (2696622.343 1245221.347)


In [6]:
def prepare_demand_data(gemeinden_zentral_gdfs, gemeinden_höhere_Dichte_gdfs, gemeinden_niedrige_Dichte_gdfs,
                        nachfrage_zentral, nachfrage_höhere_Dichte, nachfrage_niedrige_Dichte,
                        nachfrage_zentral_timestamps, nachfrage_höhere_Dichte_timestamps,
                        nachfrage_niedrige_Dichte_timestamps, Gemeinde, bahnhof_gdf, nachfrage_bahnhof_timestamps):

    # Plotten der Nachfragepunkte für Meilen
    punkte_zentral = gemeinden_zentral_gdfs.loc[nachfrage_zentral, 'geometry'].centroid
    punkte_höhere_Dichte = gemeinden_höhere_Dichte_gdfs.loc[nachfrage_höhere_Dichte, 'geometry'].centroid
    punkte_niedrige_Dichte = gemeinden_niedrige_Dichte_gdfs.loc[nachfrage_niedrige_Dichte, 'geometry'].centroid

    # Erstellen eines GeoDataFrame für die Punkte von Meilen mit Zeitstempel
    punkte_zentral_gdf = gpd.GeoDataFrame(geometry=punkte_zentral, crs=gemeinden_zentral_gdfs.crs)
    punkte_zentral_gdf['timestamp'] = nachfrage_zentral_timestamps
    punkte_zentral_gdf['gemeinde'] = Gemeinde
    punkte_höhere_Dichte_gdf = gpd.GeoDataFrame(geometry=punkte_höhere_Dichte, crs=gemeinden_höhere_Dichte_gdfs.crs)
    punkte_höhere_Dichte_gdf['timestamp'] = nachfrage_höhere_Dichte_timestamps
    punkte_höhere_Dichte_gdf['gemeinde'] = Gemeinde
    punkte_niedrige_Dichte_gdf = gpd.GeoDataFrame(geometry=punkte_niedrige_Dichte, crs=gemeinden_niedrige_Dichte_gdfs.crs)
    punkte_niedrige_Dichte_gdf['timestamp'] = nachfrage_niedrige_Dichte_timestamps
    punkte_niedrige_Dichte_gdf['gemeinde'] = Gemeinde

    # Extrahieren der Geometrie basierend auf dem Namen
    geometry = bahnhof_gdf.loc[bahnhof_gdf['name'] == Gemeinde, 'geometry'].values[0]

    # Erstellen eines neuen DataFrame mit den Timestamps und der extrahierten Geometrie
    punkte_bahnhof = pd.DataFrame({
        'timestamp': nachfrage_bahnhof_timestamps,
        'geometry': [geometry] * len(nachfrage_bahnhof_timestamps)
    })
    punkte_bahnhof_gdf = gpd.GeoDataFrame(punkte_bahnhof, geometry='geometry', crs=bahnhof_gdf.crs)
    punkte_bahnhof_gdf['timestamp'] = nachfrage_bahnhof_timestamps
    punkte_bahnhof_gdf['gemeinde'] = Gemeinde

    # Alle Punkte zusammenführen
    alle_punkte_gemeinde = pd.concat([punkte_zentral_gdf, punkte_höhere_Dichte_gdf, punkte_niedrige_Dichte_gdf, punkte_bahnhof_gdf])

    return alle_punkte_gemeinde

meilen_punkte_df = prepare_demand_data(gdf_zentrale_Dichte, gdf_hohe_Dichte, gdf_tiefe_Dichte,
                                       nachfrage_meilen_zentral_pos, nachfrage_meilen_höhere_Dichte_pos,
                                       nachfrage_meilen_niedrige_Dichte_pos, nachfrage_meilen_zentral_timestamps,
                                       nachfrage_meilen_höhere_Dichte_timestamps, nachfrage_meilen_niedrige_Dichte_timestamps,
                                       'Meilen', ODPT_stops, nachfragepunkte_bahnhof_timestamps)
print(meilen_punkte_df)


                            geometry           timestamp gemeinde
22   POINT (2692931.967 1235380.276) 2018-04-20 09:20:36   Meilen
31   POINT (2692740.146 1235478.768) 2018-04-20 09:59:44   Meilen
33   POINT (2691869.011 1235914.026) 2018-04-20 09:18:57   Meilen
39   POINT (2690856.548 1236326.424) 2018-04-20 09:08:47   Meilen
38   POINT (2692424.523 1236648.398) 2018-04-20 09:04:30   Meilen
..                               ...                 ...      ...
245  POINT (2691216.542 1236096.946) 2018-04-20 09:32:58   Meilen
246  POINT (2691216.542 1236096.946) 2018-04-20 09:50:30   Meilen
247  POINT (2691216.542 1236096.946) 2018-04-20 09:47:22   Meilen
248  POINT (2691216.542 1236096.946) 2018-04-20 09:44:29   Meilen
249  POINT (2691216.542 1236096.946) 2018-04-20 09:49:23   Meilen

[462 rows x 3 columns]


In [7]:
import numpy as np
import geopandas as gpd
import pandas as pd
import matplotlib as plt
import os
#print(1)


# Erstelle Root zum Hauptordner
ROOT_FILES = 'C:/Users/Linus/PycharmProjects/BA/'
ROOT_RESOURCE = 'src/main/resources/QGIS/Nutzungsplan/'
ROOT_DOCS = 'src/main/resources/Dokumente/'
shapefile_paths = [ROOT_FILES + 'src/test/resources/input/Gebaut.shp', 'C:/Users/Linus/Documents/W1Gebaut.shp']
# Füge die verschiedene aus dem Inputornder Geo Files zusammen und speichere sie als gdf

gemeinden_gdfs = []
geo_files_tief = ['Gebaut.shp', 'W1Gebaut.shp', 'W30.gpkg', 'W2Uster.gpkg']
geo_files_hoch = ['W2Gebaut.gpkg', 'W2.gpkg', 'W3Uster.gpkg', 'W60.gpkg']
geo_files_zentral = ['Zentrum&Gewerbe.shp', 'ZonenOe.shp', 'Zentrum.gpkg', 'ZentrumUster.gpkg', 'ZentrumEgg.gpkg']

gdf_tiefe_Dichte = create_geo_dataframe(geo_files_tief)
gdf_hohe_Dichte = create_geo_dataframe(geo_files_hoch)
gdf_zentrale_Dichte = create_geo_dataframe(geo_files_zentral)

#print(gdf_tiefe_Dichte)
#Importiere die Datensätze für das Verkehrsmodell, das Strassenetz und der Gemeindengrenzen
ROOT_RESOURCE_VERKEHRSMODELL = 'src/main/resources/Verkehrsmodell/'
DATEN_VERKEHRSMODELL = pd.read_csv(ROOT_FILES + ROOT_RESOURCE_VERKEHRSMODELL + 'KTZH_00001982_00003903(1).csv')

ROOT_RESOURCE_GEMEINDENGRENZEN = 'src/main/resources/QGIS/Gemeindegrenzen/'
DATEN_GEMEINDENGRENZEN = gpd.read_file(ROOT_FILES + ROOT_RESOURCE_GEMEINDENGRENZEN + 'Gemeindegrenzen.shp')

ROOT_RESOURCE_STRASSENNETZ = 'src/main/resources/QGIS/Strassen/'
DATEN_STRASSENNETZ = gpd.read_file(ROOT_FILES + ROOT_RESOURCE_STRASSENNETZ + 'AlleStrassen.shp')

#Analyse des Verkehrsmodell in den Gemeinden
gewuenschte_gemeinden = ['Meilen', 'Egg', 'Uster']
gewuenschte_zielnamen = ['Meilen', 'Egg', 'Uster']
gewuenschte_kategorien = ['Verkehrsaufkommen']
gewuenschte_verkehrsmittel = ['oev']
gewuenschtes_jahr = 2018
#DATENSATZ_VERKEHR = DATEN_VERKEHRSMODELL

DATEN_FILTERED = filter_data(DATEN_VERKEHRSMODELL, gewuenschtes_jahr,
                             gewuenschte_gemeinden, gewuenschte_zielnamen,
                             gewuenschte_kategorien, gewuenschte_verkehrsmittel)

# Filtern der Daten für jeden Zielort
daten_meilen = filter_target_data(DATEN_FILTERED, 'Meilen')
daten_egg = filter_target_data(DATEN_FILTERED, 'Egg')
daten_uster = filter_target_data(DATEN_FILTERED, 'Uster')


# Annahme: Vordefinierte Werte für stunden_verkehrstag und prozent_verteilung_hoch/mittel/niedrig
stunden_verkehrstag = 12
prozent_verteilung_hoch = 0.5
prozent_verteilung_mittel = 0.3
prozent_verteilung_niedrig = 0.2

Anzahl_Meilen_Egg_oev = filter_and_calculate_traffic_data(
    DATEN_VERKEHRSMODELL, 'Meilen', 'Egg', 'oev', 2018, 'Verkehrsaufkommen'
)
Anzahl_Meilen_Egg_miv = filter_and_calculate_traffic_data(
    DATEN_VERKEHRSMODELL, 'Meilen', 'Egg', 'miv', 2018, 'Verkehrsaufkommen'
)
modal_split_oev_Oetwil = filter_and_calculate_traffic_data(
    DATEN_VERKEHRSMODELL, 'Oetwil am See', 'Männedorf', 'oev', 2018, 'Modal Split'
)

neues_verkehrsaufkommen_oev_Meilen_Egg = berechne_verkehrsaufkommen(
    Anzahl_Meilen_Egg_oev, Anzahl_Meilen_Egg_miv, modal_split_oev_Oetwil)

nachfrage_meilen, nachfrage_meilen_zentral,\
    nachfrage_meilen_höhere_Dichte, nachfrage_meilen_niedrige_Dichte =\
    calculate_new_demand(neues_verkehrsaufkommen_oev_Meilen_Egg, stunden_verkehrstag,
                     prozent_verteilung_hoch, prozent_verteilung_mittel,
                     prozent_verteilung_niedrig)
nachfrage_egg, nachfrage_egg_zentral,\
    nachfrage_egg_höhere_Dichte, nachfrage_egg_niedrige_Dichte =\
    calculate_demand(daten_egg, stunden_verkehrstag,
                     prozent_verteilung_hoch, prozent_verteilung_mittel,
                     prozent_verteilung_niedrig)
nachfrage_uster, nachfrage_uster_zentral,\
    nachfrage_uster_höhere_Dichte, nachfrage_uster_niedrige_Dichte =\
    calculate_demand(daten_uster, stunden_verkehrstag,
                     prozent_verteilung_hoch, prozent_verteilung_mittel,
                     prozent_verteilung_niedrig)

start_timestamp = pd.Timestamp('2018-04-20 09:00:00')
end_timestamp = pd.Timestamp('2018-04-20 10:00:00')

(nachfrage_meilen_zentral_pos, nachfrage_meilen_zentral_timestamps), \
(nachfrage_meilen_höhere_Dichte_pos, nachfrage_meilen_höhere_Dichte_timestamps), \
(nachfrage_meilen_niedrige_Dichte_pos, nachfrage_meilen_niedrige_Dichte_timestamps) = \
    verteile_nachfragepunkte('Meilen', nachfrage_meilen_zentral,
                             nachfrage_meilen_höhere_Dichte,
                             nachfrage_meilen_niedrige_Dichte,
                             gdf_zentrale_Dichte, gdf_hohe_Dichte,
                             gdf_tiefe_Dichte, start_timestamp, end_timestamp)


(nachfrage_egg_zentral_pos, nachfrage_egg_zentral_timestamps), \
(nachfrage_egg_höhere_Dichte_pos, nachfrage_egg_höhere_Dichte_timestamps), \
(nachfrage_egg_niedrige_Dichte_pos, nachfrage_egg_niedrige_Dichte_timestamps) = \
    verteile_nachfragepunkte('Egg', nachfrage_egg_zentral,
                             nachfrage_egg_höhere_Dichte,
                             nachfrage_egg_niedrige_Dichte,
                             gdf_zentrale_Dichte, gdf_hohe_Dichte,
                             gdf_tiefe_Dichte, start_timestamp, end_timestamp)

(nachfrage_uster_zentral_pos, nachfrage_uster_zentral_timestamps), \
(nachfrage_uster_höhere_Dichte_pos, nachfrage_uster_höhere_Dichte_timestamps), \
(nachfrage_uster_niedrige_Dichte_pos, nachfrage_uster_niedrige_Dichte_timestamps) = \
    verteile_nachfragepunkte('Uster', nachfrage_uster_zentral,
                             nachfrage_uster_höhere_Dichte,
                             nachfrage_uster_niedrige_Dichte,
                             gdf_zentrale_Dichte, gdf_hohe_Dichte,
                             gdf_tiefe_Dichte, start_timestamp, end_timestamp)


print(nachfrage_meilen_zentral)
print('Daten Meilen:' , nachfrage_meilen_zentral_pos)
print('Daten Meilen:' , nachfrage_meilen_höhere_Dichte_pos)
print('Daten Egg:' , nachfrage_egg_zentral_pos)
print('Daten Uster:' , nachfrage_uster)
print('Daten Uster:' , nachfrage_uster_zentral_pos)
print('Daten Uster:' , nachfrage_uster_höhere_Dichte_pos)
print('Daten Uster:' , nachfrage_uster_niedrige_Dichte_timestamps)

# Rufe die Funktion auf, um die Daten vorzubereiten
meilen_punkte_df = prepare_demand_data(gdf_zentrale_Dichte, gdf_hohe_Dichte, gdf_tiefe_Dichte,
                                       nachfrage_meilen_zentral_pos, nachfrage_meilen_höhere_Dichte_pos,
                                       nachfrage_meilen_niedrige_Dichte_pos, nachfrage_meilen_zentral_timestamps,
                                       nachfrage_meilen_höhere_Dichte_timestamps, nachfrage_meilen_niedrige_Dichte_timestamps,
                                       'Meilen')
egg_punkte_df = prepare_demand_data(gdf_zentrale_Dichte, gdf_hohe_Dichte, gdf_tiefe_Dichte,
                                       nachfrage_egg_zentral_pos, nachfrage_egg_höhere_Dichte_pos,
                                       nachfrage_egg_niedrige_Dichte_pos, nachfrage_egg_zentral_timestamps,
                                       nachfrage_egg_höhere_Dichte_timestamps, nachfrage_egg_niedrige_Dichte_timestamps,
                                       'Egg')

uster_punkte_df = prepare_demand_data(gdf_zentrale_Dichte, gdf_hohe_Dichte, gdf_tiefe_Dichte,
                                       nachfrage_uster_zentral_pos, nachfrage_uster_höhere_Dichte_pos,
                                       nachfrage_uster_niedrige_Dichte_pos, nachfrage_uster_zentral_timestamps,
                                       nachfrage_uster_höhere_Dichte_timestamps, nachfrage_uster_niedrige_Dichte_timestamps,
                                      'Uster')

# Verwendung von aussagekräftigen Variablennamen und Inline-Kommentaren
alle_punkte_df = pd.concat([egg_punkte_df, uster_punkte_df, meilen_punkte_df], ignore_index=True)
alle_punkte_df_with_passenger_numbers = add_passenger_numbers(alle_punkte_df)

alle_punkte_df_with_passenger_numbers.to_file(ROOT_FILES + ROOT_DOCS + "Nachfrage.geojson", driver='GeoJSON')

# Plotten der Nachfrageverteilung
#plot_demand_distribution(DATEN_GEMEINDENGRENZEN, gdf_zentrale_Dichte,
                         #gdf_hohe_Dichte, gdf_tiefe_Dichte,
                         #DATEN_STRASSENNETZ, alle_punkte_df_with_passenger_numbers)
# Beispielaufruf:
new_destination_gdf = create_destination_gdf(alle_punkte_df_with_passenger_numbers, gdf_zentrale_Dichte, gdf_hohe_Dichte,
                                             gdf_tiefe_Dichte)
# Speichern der GeoDataFrame als GeoJSON-Datei
new_destination_gdf.to_file(ROOT_FILES + ROOT_DOCS + "Ziele.geojson", driver='GeoJSON')

# Anwenden der Funktion für das Hinzufügen von Zielspalten
output_gdf = add_destination_columns(alle_punkte_df_with_passenger_numbers, gdf_zentrale_Dichte,
                                      gdf_hohe_Dichte, gdf_tiefe_Dichte)

# Speichern der GeoDataFrame als GeoJSON-Datei
output_gdf.to_file(ROOT_FILES + ROOT_DOCS + "output.geojson", driver='GeoJSON')

destination_gdf = filter_null_timestamp(output_gdf)

#plot_demand_distribution(DATEN_GEMEINDENGRENZEN, gdf_zentrale_Dichte,
                         #gdf_hohe_Dichte, gdf_tiefe_Dichte,
                         #DATEN_STRASSENNETZ, destination_gdf)

TypeError: calculate_new_demand() missing 1 required positional argument: 'prozent_verteilung_niedrig'